In [1]:
import pandas as pd
import numpy as np

In [46]:
data = pd.read_csv('data/four_data.csv')

In [47]:
data = data.rename(index=str, columns = {
    'a1X': 'aFX', 
    'a1Y': 'aFY', 
    'a1Z': 'aFZ', 
    'g1X': 'gFX', 
    'g1Y': 'gFY', 
    'g1Z': 'gFZ',
    
    'a0X': 'aSX', 
    'a0Y': 'aSY', 
    'a0Z': 'aSZ', 
    'g0X': 'gSX', 
    'g0Y': 'gSY', 
    'g0Z': 'gSZ',
})

In [94]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# X = data[[
#  'acc_rs_x',
#  'acc_rs_y',
#  'acc_rs_z',
#  'gyro_rs_x',
#  'gyro_rs_y',
#  'gyro_rs_z',
#  'EMG_r']]

X = data[[
 'aSX',
 'aSY',
 'aSZ',
 'gSX',
 'gSY',
 'gSZ',
 'myo'
]]

x_scaler = MinMaxScaler((0, 1))
# x_scaler = StandardScaler()
x_scaler.fit(X)
x_scaled = x_scaler.transform(X)

In [95]:
pd.DataFrame(x_scaled).describe()

,0,1,2,3,4,5,6
count,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000,48860.000000
mean,0.596446,0.518882,0.557526,0.412326,0.354139,0.450609,0.141852
std,0.214443,0.172902,0.094671,0.104814,0.116357,0.115887,0.084501
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.467927,0.441932,0.502946,0.370925,0.271730,0.401286,0.081761
50%,0.612273,0.467264,0.562338,0.410305,0.358860,0.434951,0.125786
75%,0.760939,0.646181,0.614895,0.453262,0.411495,0.503658,0.176101
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [96]:
Y = - data['gFY'] - data['gSY']

y_scaler = MinMaxScaler((-1, 1))
y_scaler.fit(Y.reshape(-1, 1))
y_scaled = y_scaler.transform(Y.reshape(-1, 1)).T[0]

/home/arkadii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/home/arkadii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


In [97]:
pd.Series(y_scaled).describe()

count    48860.000000
mean         0.332265
std          0.214458
min         -1.000000
25%          0.264689
50%          0.323255
75%          0.436689
max          1.000000
dtype: float64

In [79]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge, SGDRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import median_absolute_error, mean_squared_error, make_scorer

models = [
    ('SGD', SGDRegressor(tol=1e-3)),
    ('Linear Regression', LinearRegression()),
    ('Elastic Net', ElasticNet()),
    ('Lasso', Lasso()),
    ('Ridge', Ridge()),
#     ('SVR (Linear)', SVR(kernel='linear')),
#     ('SVR (RBF)', SVR(kernel='rbf')),
    ('GBR (100, ls)', GradientBoostingRegressor(n_estimators=100)),
    ('GBR (200, ls)', GradientBoostingRegressor(n_estimators=200)),
    ('GBR (300, ls)', GradientBoostingRegressor(n_estimators=300)),
    ('GBR (100, huber)', GradientBoostingRegressor(n_estimators=100, loss='huber')),
    ('GBR (200, huber)', GradientBoostingRegressor(n_estimators=200, loss='huber')),
    ('GBR (300, huber)', GradientBoostingRegressor(n_estimators=300, loss='huber')),
    ('GBR (400, huber)', GradientBoostingRegressor(n_estimators=400, loss='huber')),
]

def corr_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0][1]
    

scoring = {
    'neg_mean_squared_error': 'neg_mean_squared_error', 
    'explained_variance': 'explained_variance', 
    'r2': 'r2',
    'corr_coef': make_scorer(corr_score)
}

In [80]:
import numpy as np
def cross_validate_models(models, X, Y):
    for name, model in models:
        scores = cross_validate(model, X, Y, scoring=scoring, cv=10)
        print('Model: {}'.format(name))
        print('RMSE normalized: {}'.format(np.sqrt(-scores['test_neg_mean_squared_error']).mean()))
#         print('RMSE denormalized: {}'.format(np.sqrt(-scores['test_neg_mean_squared_error']).mean()*2000/2**15))
        print('Explained Variance: {}'.format(scores['test_explained_variance'].mean()))
        print('R2 score: {}'.format(scores['test_r2'].mean()))
        print('Corr coeff: {}'.format(scores['test_corr_coef'].mean()))
        print('---------------------------------------------------------------------------')

    print('---------------------------------------------------------------------------')
    print('Finished')

In [81]:
cross_validate_models(models, X, Y)

Model: SGD
RMSE normalized: 0.9302611415714732
Explained Variance: -0.09659610028320667
R2 score: -1.0257913774023533
Corr coeff: 0.0764992874554559
---------------------------------------------------------------------------
Model: Linear Regression
RMSE normalized: 0.7604059986915208
Explained Variance: 0.012339414428644568
R2 score: -0.044936496140729275
Corr coeff: 0.11755660229470777
---------------------------------------------------------------------------
Model: Elastic Net
RMSE normalized: 0.7622794848910242
Explained Variance: 0.017722170212383825
R2 score: -0.052217177820824225
Corr coeff: 0.1175566022947078
---------------------------------------------------------------------------
Model: Lasso
RMSE normalized: 0.7666752337510954
Explained Variance: 0.01516068780659512
R2 score: -0.06942237084007775
Corr coeff: 0.11755660229470774
---------------------------------------------------------------------------
Model: Ridge
RMSE normalized: 0.7604059989229134
Explained Variance: 0